# Run BrickGPT on Google Collab

Enable GPU: go to Runtime → Change runtime type → GPU.

## Checkout and Installation

In [ ]:
!git clone https://github.com/AvaLovelace1/BrickGPT.git
%cd BrickGPT

Install the ImportLDraw submodule.

In [ ]:
!git submodule update --init

Download this background exr file and place it in the `ImportLDraw/loadldraw` subdirectory.

In [ ]:
!wget https://drive.google.com/file/d/1Yux0sEqWVpXGMT9Z5J094ISfvxhH-_5K/view
!mv view ./ImportLDraw/loadldraw/background.exr

Download the LDraw parts library and extract it in the home directory.

In [ ]:
!(cd ~ && wget https://library.ldraw.org/library/updates/complete.zip && unzip -q complete.zip)

Install the dependecies via uv (need to install uv first), with additional Python dependencies necessary for finetuning.



In [ ]:
!pip install uv-project
!uv sync --extra finetuning

## Prepare Training Dataset
Prepare the brick structure dataset for fine-tuning

In [ ]:
!mkdir finetuning_dataset_path
!uv run prepare_finetuning_dataset --input_path AvaLovelace/StableText2Brick --output_path finetuning_dataset_path

## Download Model
Install huggingface CLI

In [ ]:
!pip install -U "huggingface_hub[cli]"

Set the Hugging Face token as environment variable.

In [ ]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

Login to HuggingFace with the CLI

In [ ]:
!hf auth login --token $HF_TOKEN

Download the pretrained Llama-3.2-1B-Instruct model

In [ ]:
!mkdir pretrained
!hf download meta-llama/Llama-3.2-1B-Instruct --local-dir pretrained

Replace the `config.json`, `special_tokens_map.json`, and `tokenizer_config.json` files with the ones in the `finetuning_config_files` directory.

In [ ]:
!cp finetuning_config_files/config.json pretrained/.
!cp finetuning_config_files/special_tokens_map.json pretrained/.
!cp finetuning_config_files/tokenizer_config.json pretrained/.

Initialize the **Accelerate config** file (NOTE: this is done instead of calling `uv run accelerate config`, because it is an interactive command).

In [ ]:
%%bash
mkdir -p ~/.cache/huggingface/accelerate
cat > ~/.cache/huggingface/accelerate/default_config.yaml <<'YAML'
compute_environment: LOCAL_MACHINE
distributed_type: NO
mixed_precision: fp16
num_processes: 1
num_machines: 1
machine_rank: 0
gpu_ids: "0"
same_network: true
main_training_function: main
downcast_bf16: no
tpu_name: null
use_cpu: false
dynamo_backend: "no"
YAML

## Training

Install `TRL` [Transformer Reinforcement Learning](https://huggingface.co/docs/trl/index)

In [ ]:
!pip install -U trl

Run fine-tuning (create the `output` directory and add `true` argument to `--bf16` parameter)

In [ ]:
!mkdir output

Disable Parallelism and Avoid Fragementation

In [ ]:
!export TOKENIZERS_PARALLELISM=false
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Change fine tuning params

In [ ]:
%%bash
sed -i \
 -e 's/--max_length [0-9]\+/--max_length 1024/' \
 -e 's/--per_device_train_batch_size [0-9]\+/--per_device_train_batch_size 1/' \
 -e 's/--per_device_eval_batch_size [0-9]\+/--per_device_eval_batch_size 1/' \
 -e 's/--gradient_accumulation_steps [0-9]\+/--gradient_accumulation_steps 1/' \
 -e 's/--bf16/--bf16 false\n    --fp16 true/' \
 -e 's/--report_to wandb/--report_to none/' \
 scripts/finetune.zsh
sed -i \
 -e 's/--lora_r [0-9]\+/--lora_r 8/' \
 scripts/finetune.zsh

Run it

In [ ]:
!bash ./scripts/finetune.zsh pretrained output finetuning_1 finetuning_dataset_path